# Tabular Dataset Exercise Solution
### The dataset is comming from this [Kaggle competition](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)
### Where the goal is to predict the price of houses in King Country, USA.
#### You can check the documentation of [Autogluon](https://auto.gluon.ai/stable/index.html)

#### 1) Run Autogluon default settings on this dataset

In [3]:
### import autogluon tabular predictor and datasets
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [4]:
dataset = TabularDataset('Datasets/kc_house_data.csv')
train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=402)
print(train_data.shape)
train_data.head()

(15129, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
10549,9477201370,20150226T000000,440000.0,3,1.75,1760,8025,1.0,0,0,...,7,1230,530,1976,0,98034,47.7287,-122.192,1590,7543
15154,2770604615,20141121T000000,735000.0,4,2.00,1640,6000,1.5,0,0,...,9,1640,0,1911,0,98119,47.6505,-122.375,1900,6000
20283,662440020,20150226T000000,380000.0,4,2.50,2420,4981,2.0,0,0,...,9,2420,0,2009,0,98038,47.3785,-122.023,2420,5000
7865,3211100450,20140814T000000,217000.0,3,1.00,1400,7800,1.0,0,0,...,7,1400,0,1962,0,98059,47.4789,-122.159,1400,7800
1614,2624049117,20140903T000000,425000.0,3,1.00,1550,4160,1.5,0,0,...,6,1550,0,1926,0,98118,47.5387,-122.265,1590,5000


In [5]:
label = 'price'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    1.512900e+04
mean     5.410016e+05
std      3.701607e+05
min      7.500000e+04
25%      3.200000e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64


In [6]:
### call tabular predictor with default settings

In [ ]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20221021_131807/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221021_131807/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7700000.0, 75000.0, 541001.55523, 370160.66869)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11692.44 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.0% of available memory)


[1000]	valid_set's rmse: 135959
[2000]	valid_set's rmse: 134862


In [13]:
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2453,2111010340,20140612T000000,4,2.5,2490,8124,2.0,0,0,3,7,2490,0,2001,0,98092,47.3347,-122.170,2760,6300
9453,7215730040,20140515T000000,4,3.0,3150,9130,2.0,0,0,3,9,3150,0,2001,0,98075,47.5974,-122.017,2970,6228
809,1786640070,20140806T000000,3,2.0,1950,8698,1.0,0,0,3,8,1950,0,1999,0,98042,47.3900,-122.153,2330,7212
3109,2641800015,20141124T000000,3,1.0,960,8291,1.0,0,0,3,6,960,0,1950,0,98146,47.5006,-122.334,1110,8231
15286,7011201306,20150212T000000,4,4.0,2530,1774,3.0,0,2,3,9,2100,430,2013,0,98119,47.6362,-122.369,2160,2400


In [27]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -60684.940791756635
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -60684.940791756635,
    "root_mean_squared_error": -120239.02103770745,
    "mean_squared_error": -14457422180.106667,
    "r2": 0.8884094383188936,
    "pearsonr": 0.944365834773248,
    "median_absolute_error": -34650.0625
}


In [16]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-110553.985218,-108453.564605,0.759549,0.130541,23.855707,0.006028,0.001228,0.378422,2,True,12
1,CatBoost,-115355.709919,-124688.275829,0.013510,0.005669,13.440012,0.013510,0.005669,13.440012,1,True,6
2,NeuralNetFastAI,-116296.056824,-113839.551183,0.148357,0.038012,14.218670,0.148357,0.038012,14.218670,1,True,8
3,LightGBMXT,-117533.976782,-134772.280084,0.733818,0.098817,5.020890,0.733818,0.098817,5.020890,1,True,3
4,LightGBM,-119447.773607,-131021.079891,0.468927,0.069106,3.009331,0.468927,0.069106,3.009331,1,True,4
5,XGBoost,-125837.361380,-122627.483075,0.136237,0.022195,6.249284,0.136237,0.022195,6.249284,1,True,9
6,LightGBMLarge,-126406.216973,-128412.918788,0.413520,0.044380,3.760841,0.413520,0.044380,3.760841,1,True,11
7,RandomForestMSE,-127408.629182,-133070.626462,0.692541,0.100176,14.895086,0.692541,0.100176,14.895086,1,True,5
8,ExtraTreesMSE,-136689.896601,-140276.706106,0.542098,0.093441,4.912256,0.542098,0.093441,4.912256,1,True,7
9,NeuralNetTorch,-154699.488791,-174614.254381,0.116000,0.026973,18.280615,0.116000,0.026973,18.280615,1,True,10


#### 2) Run Autogluon with your hyperparameters

In [17]:
import autogluon.core as ag

In [24]:
eval_metric = 'mae'

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 20,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

cat_options = {
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  
                   'CAT': cat_options
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=eval_metric, problem_type='regression').fit(
    train_data, time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221018_085546/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221018_085546/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5419.72 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting Dateti

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	-72503.6479	 = Validation score   (-mean_absolute_error)
	0.48s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-71730.751	 = Validation score   (-mean_absolute_error)
	0.5s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-72378.4915	 = Validation score   (-mean_absolute_error)
	0.63s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-157264.605	 = Validation score   (-mean_absolute_error)
	0.5s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-74963.6513	 = Validation score   (-mean_absolute_error)
	0.56s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: CatBoost ... Tuning model for up to 179.97s of the 597.0s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 7481.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost/T1 ...
	-65489.3744	 = Validation score   (-mean_absolute_error)
	50.78s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost/T2 ...
	-66275.1949	 = Validation score   (-mean_absolute_error)
	13.23s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost/T3 ...
	-65217.6414	 = Validation score   (-mean_absolute_error)
	115.96s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ... Tuning model for up to 179.97s of the 416.79s of remaining time.
Fitted model: NeuralNetTorch/1517d_00000 ...
	-493458.1231	 = Validation score   (-mean_absolute_error)
	16.42s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: NeuralNetTorch/1517d_00001 ...
	-76815.7294	 = Validation score   (-mean_absolute_error)
	11.4s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: NeuralNetTorch/151

In [26]:
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [316225.78125, 725385.625, 336570.6875, 178633.625, 1012081.0625]


Evaluation: mean_absolute_error on test data: -60684.940791756635
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -60684.940791756635
}


#### 3) Run Ensembling with you hyperparameters 

In [29]:
eval_metric = 'mae'

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 20,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

cat_options = {
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  
                   'CAT': cat_options
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 15*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label,
                             eval_metric=eval_metric,
                             problem_type='regression').fit(
    train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = hyperparameters, 
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221018_090154/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221018_090154/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7700000.0, 75000.0, 541001.55523, 370160.66869)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5427.81 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.1% of available memory)
	

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	-67127.904	 = Validation score   (-mean_absolute_error)
	0.46s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	-66129.7846	 = Validation score   (-mean_absolute_error)
	0.5s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	-64947.335	 = Validation score   (-mean_absolute_error)
	0.61s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	-153437.2012	 = Validation score   (-mean_absolute_error)
	0.5s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	-69987.0678	 = Validation score   (-mean_absolute_error)
	0.55s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: CatBoost_BAG_L1/T1 ...
	-61295.9184	 = Validation score   (-mean_absolute_error)
	35.42s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T2 ...
	-62054.8306	 = Validation score   (-mean_absolute_error)
	9.65s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T3 ...
	-60176.9641	 = Validation score   (-mean_absolute_error)
	100.29s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T4 ...
	-60483.8289	 = Validation score   (-mean_absolute_error)
	124.31s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T5 ...
	-61580.7449	 = Validation score   (-mean_absolute_error)
	121.88s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ...
Fitted model: NeuralNetTorch_BAG_L1/T1 ...
	-475981.836	 = Validation score   (-mean_absolute_error)
	15.27s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: N

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L2/T1 ...
	-60863.7161	 = Validation score   (-mean_absolute_error)
	0.69s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T2 ...
	-61094.7749	 = Validation score   (-mean_absolute_error)
	0.58s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T3 ...
	-60693.6883	 = Validation score   (-mean_absolute_error)
	0.79s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T4 ...
	-134402.4679	 = Validation score   (-mean_absolute_error)
	0.6s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T5 ...
	-61162.5123	 = Validation score   (-mean_absolute_error)
	0.68s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L2 ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: CatBoost_BAG_L2/T1 ...
	-60952.8901	 = Validation score   (-mean_absolute_error)
	4.08s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T2 ...
	-61885.4244	 = Validation score   (-mean_absolute_error)
	2.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T3 ...
	-60968.1895	 = Validation score   (-mean_absolute_error)
	11.88s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T4 ...
	-60939.7042	 = Validation score   (-mean_absolute_error)
	8.08s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T5 ...
	-61017.8538	 = Validation score   (-mean_absolute_error)
	4.9s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L2 ...
Fitted model: NeuralNetTorch_BAG_L2/T1 ...
	-484087.1193	 = Validation score   (-mean_absolute_error)
	16.22s	 = Training   runtime
	0.13s	 = Validation runtime
Fitted model: NeuralNetT

In [30]:
predictor.leaderboard(extra_info=True, silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,...,child_model_type,hyperparameters,hyperparameters_fit,ag_args_fit,features,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,-61451.825713,1.129765,976.514550,0.000881,0.583586,2,True,16,7,...,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetTorch_BAG_L1/T1, CatBoost_BAG_L1/T1, LightGBM_BAG_L1/T2, CatBoost_BAG_L1/T3, NeuralNetTorch_BAG_L1/T2, CatBoost_BAG_L1/T2, CatBoost_BAG_L1/T4]",{'ensemble_size': 100},{'ensemble_size': 100},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetTorch_BAG_L1/T1, CatBoost_BAG_L1/T1, LightGBM_BAG_L1/T2, CatBoost_BAG_L1/T3, NeuralNetTorch_BAG_L1/T2, CatBoost_BAG_L1/T2, CatBoost_BAG_L1/T4]",[]
1,CatBoost_BAG_L1/T3,-62000.277814,0.115769,335.221516,0.115769,335.221516,1,True,8,24,...,CatBoostModel,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[view, sqft_living, sqft_lot15, sqft_basement, id, bathrooms, bedrooms, date.month, yr_built, date, sqft_above, sqft_living15, zipcode, condition, lat, waterfront, date.year, date.dayofweek, yr_renovated, long, date.day, floors, grade, sqft_lot]","{'iterations': 10000, 'learning_rate': 0.023861097124304616, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'MAE', 'depth': 8, 'l2_leaf_reg': 3.1795327319875875}",{'iterations': 4928},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None}",[],"[LightGBM_BAG_L2/T4, LightGBM_BAG_L2/T3, CatBoost_BAG_L2/T2, CatBoost_BAG_L2/T3, WeightedEnsemble_L2, LightGBM_BAG_L2/T2, NeuralNetTorch_BAG_L2/T4, NeuralNetTorch_BAG_L2/T1, CatBoost_BAG_L2/T4, LightGBM_BAG_L2/T1, NeuralNetTorch_BAG_L2/T3, NeuralNetTorch_BAG_L2/T5, CatBoost_BAG_L2/T1, CatBoost_BAG_L2/T5, WeightedEnsemble_L3, NeuralNetTorch_BAG_L2/T2, LightGBM_BAG_L2/T5]"
2,WeightedEnsemble_L3,-62069.377452,4.833001,1748.832119,0.000727,0.586759,3,True,32,10,...,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[LightGBM_BAG_L2/T1, CatBoost_BAG_L2/T4, NeuralNetTorch_BAG_L2/T5, LightGBM_BAG_L2/T3, CatBoost_BAG_L2/

In [31]:
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: mean_absolute_error on test data: -59758.405225844384
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -59758.405225844384
}
